In [1]:
import pandas as pd
import numpy as np
import os

# import time
import multiprocessing as mp
import glob
import sys  
sys.path.insert(0, '../')

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from muwclass_library import loo_prepare, loo_train_and_classify, loo_save_res, plot_confusion_matrix,  confident_flag, confident_sigma
from bokeh.io import show
from bokeh.io import export_png
from sklearn.metrics import confusion_matrix
from collections import Counter
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
#dir_data = '/Volumes/PoplarII/Projects/2019/MUWCLASS_Project/ML/ML_pipelines_merge/Results/MUWCLASS_Classify_evaluation' # this is old results with wrong extinction correction (use band width rather than effetive wavelength)
dir_data = '/media/orion51/PoplarII/Projects/2019/MUWCLASS_Project/ML/DATA/MUWCLASS_Classify_evaluation'
class_labels = ['AGN','CV','HM-STAR','LM-STAR','HMXB','LMXB','NS','YSO']
for test, title in zip(['final_08262022'], ['CM']):    
    files = glob.glob(f'{dir_data}/LOO_{test}_results/*/classes.csv')
    dfs = [pd.read_csv(f, usecols=range(len(class_labels))) for f in files]
    print(len(dfs))
    df_median   = pd.concat(dfs).groupby(level=0).median()
    #df_1sig_up  = pd.concat(dfs).groupby(level=0).quantile(0.84135)
    #df_1sig_low = pd.concat(dfs).groupby(level=0).quantile(0.15865)
    df_2sig_up  = pd.concat(dfs).groupby(level=0).quantile(0.97725)
    df_2sig_low = pd.concat(dfs).groupby(level=0).quantile(0.02275)
    df_mean     = pd.concat(dfs).groupby(level=0).mean()
    df_std      = pd.concat(dfs).groupby(level=0).std()


    df_class = df_median.idxmax(axis=1)
    df_prob = df_median.max(axis=1)
    #df_prob_1sig_up  = pd.DataFrame(data=[df_1sig_up.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_1sig_upp'])
    #df_prob_1sig_low = pd.DataFrame(data=[df_1sig_low.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_1sig_low'])
    df_prob_2sig_up  = pd.DataFrame(data=[df_2sig_up.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_2sig_upp'])
    df_prob_2sig_low = pd.DataFrame(data=[df_2sig_low.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_2sig_low'])
    df_class_mean = df_mean.idxmax(axis=1)
    df_prob_mean = df_mean.max(axis=1)
    df_prob_e = pd.DataFrame(data=[df_std.values[i][np.argmax(np.array(df_mean), axis=1)[i]]  for i in range(len(df_std))], columns=['Class_prob_e'])

    df_mean = df_mean.add_prefix('P_')
    df_std  = df_std.add_prefix('e_P_')
    df_median = df_median.add_prefix('P_median_')
    df_2sig_up = df_2sig_up.add_prefix('P_2sig_upp_')
    df_2sig_low = df_2sig_low.add_prefix('P_2sig_low_')

    df = pd.concat([df_median, df_2sig_up, df_2sig_low, df_mean, df_std, df_class, df_prob, df_prob_2sig_up, df_prob_2sig_low, df_class_mean, df_prob_mean, df_prob_e, pd.read_csv(files[0]).name, pd.read_csv(files[0]).true_Class],axis=1).rename(columns={0:'Class_median',1:'Class_prob_median',2:'Class',3:'Class_prob'})
    #print(df.columns)
   
    df_default = confident_flag(df, method = 'sigma-mean', class_cols=class_labels)
    df_default = confident_sigma(df, class_cols=class_labels)
    df_default.to_csv(f'{dir_data}/LOO_{test}_results/LOO_classes.csv',index=False)
   

    print('accuracy for all classifications:',accuracy_score(df_default.true_Class, df_default.Class))
    print('accuracy for confident classifications:',accuracy_score(df_default.loc[df_default.conf_flag>0, 'true_Class'], df_default.loc[df_default.conf_flag>0, 'Class']))
    print('balanced accuracy for all classifications:',balanced_accuracy_score(df_default.true_Class, df_default.Class))
    print('balanced accuracy for confident classifications:',balanced_accuracy_score(df_default.loc[df_default.conf_flag>0, 'true_Class'], df_default.loc[df_default.conf_flag>0, 'Class']))
   
   
   
    CM = plot_confusion_matrix(df_default, title=f'recall {title}',classes=class_labels)
    #show(CM)
    conf_CM = plot_confusion_matrix(df_default[df_default.conf_flag>0], title=f'confident recall {title}',classes=class_labels,count_fraction=True,df_all = df_default)
    CM_p = plot_confusion_matrix(df_default, title=f'precision {title}',classes=class_labels,cm_type='precision')
    #show(CM)
    conf_CM_p = plot_confusion_matrix(df_default[df_default.conf_flag>0], title=f'confident precision {title}',classes=class_labels,cm_type='precision',count_fraction=True,df_all = df_default)
    export_png(CM, filename=f'./figs/{title}.png')
    export_png(conf_CM, filename=f'./figs/{title}_conf.png')
    export_png(CM_p, filename=f'./figs/{title}_precision.png')
    export_png(conf_CM_p, filename=f'./figs/{title}_conf_precision.png')

1000
accuracy for all classifications: 0.8860931655899354
accuracy for confident classifications: 0.9701931150293871
balanced accuracy for all classifications: 0.7022678504741028
balanced accuracy for confident classifications: 0.7900553872136309


In [5]:
df_default = pd.read_csv(f'./LOO_classes.csv')
   

print('accuracy for all classifications:',accuracy_score(df_default.true_Class, df_default.Class))
print('accuracy for confident classifications:',accuracy_score(df_default.loc[df_default.conf_flag>0, 'true_Class'], df_default.loc[df_default.conf_flag>0, 'Class']))
print('balanced accuracy for all classifications:',balanced_accuracy_score(df_default.true_Class, df_default.Class))
print('balanced accuracy for confident classifications:',balanced_accuracy_score(df_default.loc[df_default.conf_flag>0, 'true_Class'], df_default.loc[df_default.conf_flag>0, 'Class']))
print('f1_score for all classifications:',f1_score(df_default.true_Class, df_default.Class, average='macro'))
print('f1_score for confident classifications:',f1_score(df_default.loc[df_default.conf_flag>0, 'true_Class'], df_default.loc[df_default.conf_flag>0, 'Class'], average='macro'))


print('completeness:',len(df_default.loc[df_default.conf_flag>0])/len(df_default))
class_labels = ['AGN','CV','HM-STAR','HMXB','LM-STAR','LMXB','NS','YSO']
class_completeness = [len(df_default.loc[(df_default.conf_flag>0) & (df_default.Class==c)]) /len(df_default.loc[(df_default.Class==c)]) for c in class_labels ]
print('balanced completeness:',np.mean(class_completeness))
 

accuracy for all classifications: 0.8860931655899354
accuracy for confident classifications: 0.9701931150293871
balanced accuracy for all classifications: 0.7022678504741028
balanced accuracy for confident classifications: 0.7900553872136309
f1_score for all classifications: 0.6811556981396667
f1_score for confident classifications: 0.7949102762120428
completeness: 0.8099285957157429
balanced completeness: 0.5400939092091234
